In [2]:
# import scipy
import scipy.optimize

from math import pi
from math import sqrt
from math import sin

import numpy as np
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

#import Ipynb_importer
#import Chiara as H55

from pyDOE import *
import numpy as np


In [38]:
def sizing_code():
# Design variables
# ---
    J_SL = 0.5              #current density [0, 1] 0 the motor is off, 1 full power
    d_over_l_SL = 0.5       #form factor [0.1,1]*2 <=> [0.2,2]

#Propeller parameters
# ---

    RPM = 2500
    n = RPM/60
    R = 1                              #PROPELLER RADIUS
    D = 2*R
    omega = n*2*np.pi
   # Power = 211084                      #Max power
   # Thrust = 3187.8                     #Max Thrust
   # Torque = Power/(2*np.pi*n)
    
    Torque = 555.25                      # RMS of the torque (Power/(2*pi*n))  From excel file  
    Power = Torque*(2*np.pi*n)           # Power and torque in the cruise phase (most lasting phase) 

    #Reference motor SIEMENS SP-260D
    Torque_ref = 1000                                                                       # (Nm)
    RPM_ref = 3750                                                                          # (Tr/min) Optimum RPM
    Pj_ref = 10000                                                                          # Joule losses (W)
    Pi_ref = 6000                                                                           # Iron losses (W)
    Pc_ref = 16000                                                                          # Total losses (W)
    m_motor_ref = 50                                                                        # Motor mass (kg)
    d_ref = 0.418                                                                           # Diameter (m)
    l_ref = 0.300                                                                           # Length (m)
    Power_ref = 261000                                                                      # Motor power (W)
    f_ref = 750
    
    #Motor Scaling law (_SL)
    # ---
    eff_SL = 1
    eff_SL_last = 0
    k = 0
    while(abs(eff_SL-eff_SL_last) > 0.0001): #loop that converge to the efficiency value of the propeller - WHEN IT CONVERGES

        k = k+1
        Power_corr = Power*(2-eff_SL) #real amount of power needed taking into accont the motor efficiency
        Torque_SL = Torque/Torque_ref
        Power_SL = Power_corr/Power_ref
        omega_SL = Power_SL/Torque_SL

        omega_ref = 2*np.pi*RPM_ref/60
        Omega = omega_SL*omega_ref

        RPM_new = Omega*60/(2*np.pi)

        #Hypothesis
        B_SL = 1
        m_iron_SL = 1
        d_SL = ((Torque_SL*d_over_l_SL)/(J_SL*B_SL))**(1/3)
        l_SL = d_SL/d_over_l_SL

        #parameters
        m_motor_SL = d_SL*l_SL
        f_SL = omega_SL*d_SL
        Pi_SL = (f_SL**1.5)*m_iron_SL   #FROM IMECHE
        Pj_SL = (J_SL**2)*d_SL*l_SL     #FROM IMECHE
        Pc = Pj_SL*Pj_ref + Pi_SL*Pi_ref
        Pc_SL = Pc/Pc_ref
        kc_SL = Pc_SL/(d_SL*l_SL)
        
        #keep last efficiency value and finding the new one for the loop
        eff_SL_last = eff_SL
        eff_SL = (Power_corr-Pc)/Power_corr
        
    
    
    #PRINT THE RESULTS IN THE TABLE
    col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']
    df = pd.DataFrame()
    df = df.append([{'Type': 'Motor', 'Name': 'Iterations', 'Value': k, 'Unit': '[-]', 'Comment': 'power'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Propeller power', 'Value': Power, 'Unit': '[W]', 'Comment': 'power'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Propeller torque', 'Value': Torque, 'Unit': '[Nm]', 'Comment': 'Torque'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'RPM', 'Value': RPM_new, 'Unit': '[tr/min]', 'Comment': 'RPM'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Magnetic field', 'Value': B_SL, 'Unit': '[Tesla]', 'Comment': 'Magnetic field / Magnetic field ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Current density', 'Value': J_SL, 'Unit': '[A/m^2]', 'Comment': 'j / j_ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'D/L', 'Value': d_over_l_SL, 'Unit': '[-]', 'Comment': 'Form factor'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'omega_new', 'Value': Omega, 'Unit': '[rad/s]', 'Comment': 'omega new'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Motor Diameter', 'Value': d_SL*d_ref, 'Unit': '[m]', 'Comment': 'Diameter / Diameter ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Motor Length', 'Value': l_SL*l_ref, 'Unit': '[m]', 'Comment': 'Length / Length ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Frequency', 'Value': f_SL*f_ref, 'Unit': '[Hz]', 'Comment': 'Frequency / Frequency ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Joule losses', 'Value': Pj_SL*Pj_ref, 'Unit': '[W]', 'Comment': 'Pj / Pj ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Iron losses', 'Value': Pi_SL*Pi_ref, 'Unit': '[W]', 'Comment': 'Pi / P ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Cooling power', 'Value': Pc_SL*Pc_ref, 'Unit': '[W]', 'Comment': 'Pc / Pc ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Cooling system efficiency', 'Value': kc_SL, 'Unit': '[-]', 'Comment': 'kc / kc ref'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Motor power', 'Value': Power_corr, 'Unit': '[W]', 'Comment': 'Power corrected by the motor efficiency'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Efficiency', 'Value': eff_SL, 'Unit': '[-]', 'Comment': 'Efficiency of the motor'}])[col_names]
    df = df.append([{'Type': 'Motor', 'Name': 'Motor mass', 'Value': m_motor_SL*m_motor_ref, 'Unit': '[kg]', 'Comment': 'm_motor / m_motor ref'}])[col_names]
    items = sorted(df['Type'].unique().tolist())
    def f(Type):
         return df[df['Type']==Type] 
    widgets.interact(f, Type=items)
    return f

In [39]:
sizing_code()

C:\Users\Chiara\AppData\Local\Temp\ipykernel_7336\1811478786.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([{'Type': 'Motor', 'Name': 'Iterations', 'Value': k, 'Unit': '[-]', 'Comment': 'power'}])[col_names]
C:\Users\Chiara\AppData\Local\Temp\ipykernel_7336\1811478786.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([{'Type': 'Motor', 'Name': 'Propeller power', 'Value': Power, 'Unit': '[W]', 'Comment': 'power'}])[col_names]
C:\Users\Chiara\AppData\Local\Temp\ipykernel_7336\1811478786.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([{'Type': 'Motor', 'Name': 'Propeller torque', 'Value': Torque, 'Unit': '[Nm]', 'Comment': 'Torque'}])[col_names]
C:\Users\Chiara\AppDa

interactive(children=(Dropdown(description='Type', options=('Motor',), value='Motor'), Output()), _dom_classes…

<function __main__.sizing_code.<locals>.f(Type)>